# Интерфейсы scikit-learn


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from numpy.typing import NDArray
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
import yaml

with open("../../config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

## Estimator

Для примера построим простой estimator, который в перспективе будет вычитать из признаков их среднее значение и после сдвигать признаки на заранее заданную константу


In [ ]:
class SubtractMeanAndShiftEstimator(BaseEstimator):
    def __init__(self, shift=0.0):
        self.shift: float = shift
        self.means_: NDArray = (
            None  # we add a trailing underscore for parameters which will be learnt in fit()
        )

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

In [ ]:
m = SubtractMeanAndShiftEstimator(shift=3)

Метод `get_params()` реализован в `BaseEstimator`, и мы можем сразу использовать его для получения гиперпараметров модели. Это возможно, так как единственный гиперпараметр `shift` был передан как явное ключевое слово в контрукторе

Обратите внимание, что соответствующий аттрибут класса должен совпадать с ключевым словом: `self.shift = shift`


In [ ]:
m.get_params()

Аналогично мы можем использовать `set_params()` для задания значений гиперпараметров. Этот метод пригодится при поиске оптимальных значений гиперпараметров


In [ ]:
m.set_params(shift=5)
m.get_params()

In [ ]:
X = np.array(
    [
        [1, 10],
        [3, 30],
        [2, 20],
    ]
)
y = np.array(
    [
        [0, -8],
        [2, 10],
        [1, 1],
    ]
)
m.fit(X, y)

В sklearn есть класс sklearn.base.OutlierMixin, который позволяет реализовывать кастомные классы для определения выбросов.
Он добавляет:

-   атрибут \_estimator_type, по умолчанию outlier_detector
-   fit_predict.

Метод fit() работает в формате без учителя, predict же должен классифицировать данные на аутлаеры (возвращать для них -1) и обыычные данные (возвращать 1). Для классификации используется отсечка по порогу предсказаний, полученных внутренним.
Во встроенных методах функция оценки доступна с помощью метода `score_samples`, в то время как порог можно задать параметром `contamination`.
Например, для гауссовских данных можно использовать sklearn.covariance.EllipticEnvelope.


**Задание**: Создайте свой эстиматор с использованием sklearn.base.OutlierMixin, который будет определять выбросы на основе интерквартильного размаха.
Он должен возвращать один столбец с 1 и -1, а также позволять задавать порог для квантиля, определяющего размах. Не забудьте, что он должен быть двухсторонним.
Ваш эстиматор должен работать и для датафреймов, и для numpy массивов.


In [ ]:
from sklearn.base import BaseEstimator, OutlierMixin


class MyEstimator(BaseEstimator, OutlierMixin):
    def __init__(self, factor=1.5):
        self.factor = factor

    def fit(self, X, y=None):
        self.is_fitted_ = True
        X = pd.DataFrame(X.copy())
        q1 = X.quantile(0.35)
        q3 = X.quantile(0.55)
        iqr = q3 - q1
        self._lower_bound = q1 - (self.factor * iqr)
        self._upper_bound = q3 + (self.factor * iqr)
        print(X.min(axis=0))
        return self

    def predict(self, X):
        temp = np.ones(len(X))
        temp[
            (X > self._lower_bound.values).all(axis=1)
            & (X < self._upper_bound.values).all(axis=1)
        ] = -1
        return temp

In [ ]:
estimator = MyEstimator()
true_cov = np.array([[0.8, 0.3], [0.3, 0.4]])
X = np.random.RandomState(42).multivariate_normal(mean=[0, 0], cov=true_cov, size=500)
e = estimator.fit(X)
# pred = estimator.predict(X)
e._lower_bound

In [ ]:
e._upper_bound

In [ ]:
e.predict(X)

## Predictor

Рассмотрим тот же класс, но добавим к нему методы `predict()` и `score()`


In [ ]:
class SubtractMeanAndShiftPredictor(BaseEstimator):
    def __init__(self, shift=0.0):
        self.shift: float = shift
        self.means_: NDArray = (
            None  # we add a trailing underscore for parameters which will be learnt in fit()
        )

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def predict(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X - e @ self.means_.reshape(-1, 1).T + self.shift

    def score(self, X: NDArray, y: NDArray) -> float:
        return r2_score(
            y, self.predict(X)
        )  # R2 \in (-\infty; 1] is the coefficient of determination

Так как мы специально добавили небольшое отклонение в y, наш R2 чуть меньше 1


In [ ]:
model = SubtractMeanAndShiftPredictor(shift=1)
model.fit(X)
model.predict(X)
# model.score(X, y)

## Transformer

Рассмотрим тот же класс, но добавим к нему метод `transform()`


In [ ]:
class SubtractMeanAndShiftTransformer(
    BaseEstimator, OneToOneFeatureMixin, TransformerMixin
):

    def __init__(self, shift=0.0):
        self.shift: float = shift
        self.means_: NDArray = (
            None  # we add a trailing underscore for parameters which will be learnt in fit()
        )

    def fit(self, X: NDArray, y: NDArray = None):
        # y is ignored here
        self.means_ = X.mean(axis=0)  # the first axis corresponds to samples by default
        return self

    def transform(self, X: NDArray) -> NDArray:
        e = np.ones((X.shape[0], 1))
        return X - e @ self.means_.reshape(-1, 1).T + self.shift

In [ ]:
t = SubtractMeanAndShiftTransformer(shift=5)
t.fit(X)
t.transform(X)

Так как мы добавили `TransformerMixin`, мы можем использовать метод `fit_transform()`, не реализуя его явно


In [ ]:
t.fit_transform(X)

Аналогично мы можем использовать метод `get_feature_names_out()`, так как мы добавили `OneToOneFeatureMixin`


In [ ]:
# t.get_feature_names_out(input_features=["x", "y"])

Используем датасет с домами как пример. Вспомним, что мы делали в прошлый раз, и попробуем заполнить пропущенные значения в некоторых числовых столбцах.
Для этого используем трансформер по столбцам.


In [ ]:
df = pd.read_csv(cfg["house_prices"]["train_dataset"])
df.head()

По умолчанию, только указанные столбцы трансформируются и возвращаются (remainder=`drop`). Мы же сделаем так, чтобы все остальные столбцы тоже возвращались, просто с ними бы ничего не делалось.


In [ ]:
ct = ColumnTransformer(
    [
        (
            "mean_impute",
            SimpleImputer(strategy="mean"),
            ["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea"],
        )
    ],
    remainder="passthrough",
)

ct.fit(df)

In [ ]:
ct.transform(df)

In [ ]:
ct.set_output(transform="pandas")

Если у датасета появятсся столбцы, которые не были представлены во время fit (даже среди тех, что не трансформировались), то они будут выкинуты на этапе transform.


In [ ]:
df["temp"] = 0
ct.transform(df)

**Задание**: Перейдите к медианному заполнению пропусков. Проверьте, что результаты, полученные с помощью трансформации, соответствуют преобразованию напрямую.


In [ ]:
ct = ColumnTransformer(
    [
        (
            "median_impute",
            SimpleImputer(strategy="median"),
            ["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea"],
        )
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)

ct.set_output(transform="pandas")
df_transform = ct.fit_transform(df)

df_copy = df.copy()
numeric_cols = ["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea"]
df_copy[numeric_cols] = df_copy[numeric_cols].fillna(df[numeric_cols].median())

df_transform = df_transform[df_copy.columns]

print(df_transform.index.equals(df_copy.index))
print(df_transform.columns.equals(df_copy.columns))

df_transform.loc[0, "Id"] = 10
diff = df_transform.compare(df_copy)
print(diff)

**Задание**: Добавьте еще нормализатор для LotFrontage, LotArea и запустите в ColumnTransformer. Обучите его и примените.


In [ ]:
from sklearn.preprocessing import StandardScaler

ct = ColumnTransformer(
    [
        (
            "median_impute",
            SimpleImputer(strategy="median"),
            ["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea"],
        ),
        ("normalize", StandardScaler(), ["LotFrontage", "LotArea"]),
    ],
    remainder="passthrough",
    verbose_feature_names_out=True,
)

ct.set_output(transform="pandas")
df_transform = ct.fit_transform(df)

df_copy = df.copy()
df_copy[
    [
        "median_impute__SalePrice",
        "median_impute__LotArea",
        "median_impute__WoodDeckSF",
        "median_impute__MasVnrArea",
    ]
] = df_copy[["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea"]].fillna(
    df[["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea"]].median()
)

scaler = StandardScaler()
df_copy[["normalize__LotFrontage", "normalize__LotArea"]] = scaler.fit_transform(
    df_copy[["LotFrontage", "LotArea"]]
)

to_change = ["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea", "LotFrontage"]
to_remain = [col for col in df.columns if col not in to_change]
df_copy[[f"remainder__{col}" for col in to_remain]] = df_copy[to_remain]
df_copy = df_copy.drop(df.columns, axis=1)

print(df_transform.index.equals(df_copy.index))
print(df_transform.columns.equals(df_copy.columns))

df_transform = df_transform.astype(df_copy.dtypes)
print(df_transform.equals(df_copy))

df_transform = df_transform[df_copy.columns]
df_transform.loc[0, "Id"] = 10
print(df_transform.equals(df_copy))

Sklearn располагает большим количеством встроенных трансформеров. Соответствующие трансформеры есть и для категориальных фичей (более подробно рассмотрим этот тип чуть позже). Например, известное нам бинарное кодирование можно проводить с помощью OneHotEncoder()


In [ ]:
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    transformers=[
        (
            "median_impute",
            SimpleImputer(strategy="mean"),
            ["SalePrice", "LotArea", "WoodDeckSF", "MasVnrArea"],
        ),
        (
            "one_hot_encode",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            ["MSZoning", "SaleType", "SaleCondition"],
        ),
    ],
    remainder="passthrough",
)

ct.fit(df)

In [ ]:
ct.set_output(transform="pandas")
ct.transform(df)

Для выбора столбцов можно создавать make_selector, например, по выбору численных и категориальных значений.

**Доп. задание**. Сделайте трансформер для OneHotEncoder на основе make_selector так, чтобы выбирать все нечисловые столбцы. Сколько столбцов получается после трансформации?


In [ ]:
from sklearn.compose import make_column_selector as selector

# Your code

## Pipelines

С помощью Pipeline мы можем производить последовательную обработку данных и выполнять предсказание в конце


In [ ]:
X = np.array(
    [
        [1, 10],
        [3, 30],
        [2, 20],
    ]
)
y = np.array(
    [
        [0],
        [2],
        [1],
    ]
)

pipeline = Pipeline(
    [
        ("shifter", SubtractMeanAndShiftTransformer(shift=5)),
        ("regressor", LinearRegression()),
    ]
)

pipeline.fit(X, y)
y_pred = pipeline.predict(X)
print(y_pred)

Pipeline хранит последовательные Estimators в аттрибуте `steps`


In [ ]:
pipeline.steps

Перейти к объекту i-го Estimator можно напрямую через `pipeline[i]`:


In [ ]:
pipeline[0]

In [ ]:
pipeline[1].coef_

Так как Pipeline сам является Estimator, мы можем увидеть список его параметров:


In [ ]:
pipeline.get_params()

Видно, параметры промежуточных Estimator указаны как `<estimator>__<parameter>`. Следовательно, мы можем изменить параметры любого промежуточного Estimator:


In [ ]:
pipeline.set_params(shifter__shift=10)
pipeline.get_params()

**Задание**: Создайте пайплайн по преобразованию численных столбцов, содержащий импьютер и скейлер.


In [ ]:
pipeline = Pipeline(
    [("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)

transformer = ColumnTransformer(
    transformers=[("num", pipeline, selector(dtype_include="number"))],
    remainder="passthrough",
)

transformer.set_output(transform="pandas")
transformer.fit(df)
df_transform = transformer.transform(df)

print(df_transform)

**Задание**: Создайте новый трансформер, который для категориальных столбцов будет заполнять пропущенные значения наиболее часто встречаюмшщимся или новой категорией (сделайте параметром).


In [ ]:
class CategoricalImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy="most_frequent", fill_value="Unknown"):
        self.strategy = strategy
        self.fill_value = fill_value
        self.imputer = None
        self.feature_names_out_ = None

    def fit(self, X, y=None):
        if self.strategy == "most_frequent":
            self.imputer = SimpleImputer(strategy="most_frequent")
        else:
            self.imputer = SimpleImputer(
                strategy="constant", fill_value=self.fill_value
            )

        self.imputer.fit(X)
        self.feature_names_out_ = X.columns
        return self

    def transform(self, X):
        X_imputed = self.imputer.transform(X)
        return pd.DataFrame(X_imputed, columns=X.columns)

    def get_feature_names_out(self, input_features=None):
        return np.array(self.feature_names_out_)


categorical_pipeline = Pipeline(
    [
        ("imputer", CategoricalImputer(strategy="constant", fill_value="Unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

transformer = ColumnTransformer(
    transformers=[("cat", categorical_pipeline, selector(dtype_exclude="number"))],
    remainder="passthrough",
)

df_transform = transformer.fit_transform(df)
df_transform = pd.DataFrame(df_transform, columns=transformer.get_feature_names_out())
print(df_transform)

categorical_pipeline_most_frequent = Pipeline(
    [
        ("imputer", CategoricalImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

transformer = ColumnTransformer(
    transformers=[
        ("cat", categorical_pipeline_most_frequent, selector(dtype_exclude="number"))
    ],
    remainder="passthrough",
)

df_transform = transformer.fit_transform(df)
df_transform = pd.DataFrame(df_transform, columns=transformer.get_feature_names_out())
print(df_transform)

**Задание**: Создайте пайплайн по преобразованию категориальных столбцов, содержащий ваш импьютер и OneHotEncoder.


In [ ]:
categorical_pipeline = Pipeline(
    [
        ("imputer", CategoricalImputer(strategy="constant", fill_value="Unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

transformer = ColumnTransformer(
    transformers=[("cat", categorical_pipeline, selector(dtype_exclude="number"))],
    remainder="passthrough",
)

df_transform = transformer.fit_transform(df)
df_transform = pd.DataFrame(df_transform, columns=transformer.get_feature_names_out())
print(df_transform.head())

**Задание**: Создайте ColumnTransformer, который будет содержать в себе два вышеуказанных пайплайна.


In [ ]:
numeric_pipeline = Pipeline(
    [("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)

categorical_pipeline = Pipeline(
    [
        ("imputer", CategoricalImputer(strategy="constant", fill_value="Unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, selector(dtype_include="number")),
        ("cat", categorical_pipeline, selector(dtype_exclude="number")),
    ],
    remainder="passthrough",
)

transformer.set_output(transform="pandas")
df_transform = transformer.fit_transform(df)
print(df_transform.head())

**Доп.задание**: Используйте для комбинации результатов двух отдельных трансформеров FeatureUnion
